In [32]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request
import re
#import urllib2.request
import os
import http.cookiejar
import json

In [37]:
def get_soup(url,header):
    return BeautifulSoup(urllib.request.urlopen(urllib.request.Request(url,headers=header)),'html.parser')


query = input("Image for searching: ")# you can change the query for the image  here
image_type="ActiOn"
query= query.split()
query='+'.join(query)
url="https://www.google.co.in/search?q="+query+"&source=lnms&tbm=isch"
print(url)
#add the directory for your image here
DIR="Pictures"
header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
}
soup = get_soup(url,header)


ActualImages=[]# contains the link for Large original images, type of  image
for a in soup.find_all("div",{"class":"rg_meta"}):
    link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
    ActualImages.append((link,Type))

print("there are total" , len(ActualImages),"images")

if not os.path.exists(DIR):
            os.mkdir(DIR)
DIR = os.path.join(DIR, query.split()[0])

if not os.path.exists(DIR):
            os.mkdir(DIR)
###print images
for i , (img , Type) in enumerate( ActualImages):
    try:
        req = urllib.request.Request(img, headers={'User-Agent' : header})
        raw_img = urllib.request.urlopen(req).read()

        cntr = len([i for i in os.listdir(DIR) if image_type in i]) + 1
        print(cntr)
        if len(Type)==0:
            f = open(os.path.join(DIR , image_type + "_"+ str(cntr)+".jpg"), 'wb')
        else :
            f = open(os.path.join(DIR , image_type + "_"+ str(cntr)+"."+Type), 'wb')


        f.write(raw_img)
        f.close()
    except Exception as e:
        print("could not load : "+img)
        print(e)

query imagecover letter
https://www.google.co.in/search?q=cover+letter&source=lnms&tbm=isch
there are total 100 images
could not load : https://d.novoresume.com/images/doc/modern-cover-letter-template.png
expected string or bytes-like object
could not load : https://career.virginia.edu/sites/career2018.virginia.edu/files/styles/resume_thumb_325x415/public/CoverLetter_Thomas_Browne.jpg?itok=wNlcrNHe
expected string or bytes-like object
could not load : https://resumegenius.com/wp-content/uploads/2016/03/Park-Cover-Letter-Template-Black-White-233x300.jpg
expected string or bytes-like object
could not load : https://studentaffairs.unt.edu/sites/default/files/career-center/images/Sample%20Referral%20Letter.png
expected string or bytes-like object
could not load : https://resumegenius.com/wp-content/uploads/2016/03/Milano-Cover-Letter-Template-Gray-233x300.jpg
expected string or bytes-like object
could not load : https://career.virginia.edu/sites/career2018.virginia.edu/files/styles/resume_

could not load : https://go.sdsu.edu/student_affairs/career/images/coverletter.jpg
expected string or bytes-like object
could not load : https://venngage-wordpress.s3.amazonaws.com/uploads/2018/04/cover-letter-template-10.png
expected string or bytes-like object
could not load : https://www.jobscan.co/img/content/Cover-Letter-Formats-image00.png
expected string or bytes-like object
could not load : https://cdn-images.zety.com/pages/cover_letter_and_resume_template2.jpg
expected string or bytes-like object
could not load : https://www.wikihow.com/images/thumb/d/df/Cover-Letter-for-Employment.png/340px-Cover-Letter-for-Employment.png
expected string or bytes-like object
could not load : https://www.vpul.upenn.edu/careerservices/files/Anatomy_of_a_Cover_Letter_resize.JPG
expected string or bytes-like object
could not load : https://www.pomona.edu/sites/default/files/styles/inset/public/images/insets/cdo-cover-letter-format.jpg?itok=7h_dURDQ
expected string or bytes-like object
could not l